Let us take an example of Titanic problem and perform feature engineering in this problem.

In [32]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
df_train = pd.read_csv('Titanic- Machine learning from Disaster/train.csv')
df_test = pd.read_csv('Titanic- Machine learning from Disaster/test.csv')
print(df_train.shape,df_test.shape)

(891, 12) (418, 11)


In [16]:
train  = df_train.copy()
test = df_test.copy()
test['Survived']=0
concat = train.append(test,ignore_index=True)
print(concat.shape)
concat.head(2)

(1309, 12)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1,PC 17599


# Feature Engineering
### Lets add following Feature:
1. FamilySize - Parch + SibSp
2. Title - Name second word
3. FamilyId - Every family member has same surname and family size

In [21]:
# FamilySize
concat['FamilySize'] = concat.Parch + concat.SibSp
concat[concat.FamilySize>8]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,FamilySize
159,NaN,NaN,S,69.55,"Sage, Master. Thomas Henry",2,160,3,male,8,0,CA. 2343,10
180,NaN,NaN,S,69.55,"Sage, Miss. Constance Gladys",2,181,3,female,8,0,CA. 2343,10
201,NaN,NaN,S,69.55,"Sage, Mr. Frederick",2,202,3,male,8,0,CA. 2343,10
324,NaN,NaN,S,69.55,"Sage, Mr. George John Jr",2,325,3,male,8,0,CA. 2343,10
792,NaN,NaN,S,69.55,"Sage, Miss. Stella Anna",2,793,3,female,8,0,CA. 2343,10
846,NaN,NaN,S,69.55,"Sage, Mr. Douglas Bullen",2,847,3,male,8,0,CA. 2343,10
863,NaN,NaN,S,69.55,"Sage, Miss. Dorothy Edith ""Dolly""",2,864,3,female,8,0,CA. 2343,10
1079,NaN,NaN,S,69.55,"Sage, Miss. Ada",2,1080,3,female,8,0,CA. 2343,10
1233,NaN,NaN,S,69.55,"Sage, Mr. John George",9,1234,3,male,1,0,CA. 2343,10
1251,14.5,NaN,S,69.55,"Sage, Master. William Henry",2,1252,3,male,8,0,CA. 2343,10


In [29]:
#Title
Names = concat.Name.str.split('[,.]')
concat['Title']=[str.strip(Name[1]) for Name in Names]
print(concat.Title.unique())
concat.head(2)

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,FamilySize,Title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0,A/5 21171,1,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1,PC 17599,1,Mrs


In [33]:
concat.Title[concat.Title.isin(['Mme','Mlle'])]='Mlle'
concat.Title.values[concat.Title.isin(['Capt', 'Don', 'Major', 'Sir'])] = 'Sir'
concat.Title.values[concat.Title.isin(['Dona', 'Lady', 'the Countess', 'Jonkheer'])] = 'Lady'
concat.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mlle', 'Ms',
       'Major', 'Lady', 'Sir', 'Col', 'Capt', 'the Countess', 'Jonkheer',
       'Dona'], dtype=object)

In [37]:
from sklearn import preprocessing as prep
le = prep.LabelEncoder()
le.fit(concat.Title)
title = le.transform(concat.Title)
concat.Title = title
concat.Title.unique()

array([11, 12,  9,  8,  2, 14,  4, 10, 13,  7,  6, 15,  1,  0, 16,  5,  3])

In [39]:
concat.Embarked.fillna('0',inplace=True)
le.fit(concat.Embarked)
embarked = le.transform(concat.Embarked)
concat.Embarked = embarked
concat.Embarked.unique()

array([3, 1, 2, 0])

In [56]:
concat['Surname']=[str.strip(Name[0]) for Name in Names]
concat['FamilyId']=concat.Surname.str.cat(concat.FamilySize.astype(str))
concat.FamilyId.values[concat.FamilySize.values <= 2] = 'Small'
concat.FamilyId.value_counts()

Small             1184
Sage10              11
Andersson6           9
Goodwin7             8
Asplund6             7
Fortune5             6
Skoog5               6
Rice5                6
Panula5              6
Ford4                5
Ryerson4             5
Lefebre4             5
Palsson4             5
Becker3              4
Baclini3             4
Dean3                4
Allison3             4
Carter3              4
Herman3              4
Laroche3             4
Johnston3            4
West3                4
Hocking3             2
Jacobsohn3           1
Renouf3              1
Backstrom3           1
Kink-Heilmann4       1
Hocking4             1
Vander Planke3       1
Davidson3            1
Richards5            1
Name: FamilyId, dtype: int64